In [131]:
import requests
from bs4 import BeautifulSoup
import urllib.request
from collections import defaultdict
import simplejson as json
import random
import io
import gzip
import random
import os
import re

In [18]:
def getSitemap(url):
    
    '''
    A method that obtains all the gz files from the given url

    url - the sitemap url for apkpure
    '''
    
    url = requests.get(url).text
    soup = BeautifulSoup(url, 'lxml')

    li = []

    for elem in soup.find_all('loc'):
        li.append(elem.text)

    return li

#gzlist = getSitemap('https://apkpure.com/sitemap.xml')


In [20]:
# response = requests.get(url)
# compressed_file = io.BytesIO(response.content)
# decompressed_file = gzip.GzipFile(fileobj=compressed_file)
# OUTFILE_PATH = 'my_new_app.xml'
# with open(OUTFILE_PATH, 'wb') as outfile:
#     outfile.write(decompressed_file.read())

gzlist

['https://apkpure.com/sitemaps/default.xml.gz',
 'https://apkpure.com/sitemaps/topics.xml.gz',
 'https://apkpure.com/sitemaps/tag.xml.gz',
 'https://apkpure.com/sitemaps/tag-1.xml.gz',
 'https://apkpure.com/sitemaps/tag-2.xml.gz',
 'https://apkpure.com/sitemaps/group.xml.gz',
 'https://apkpure.com/sitemaps/group-1.xml.gz',
 'https://apkpure.com/sitemaps/group-2.xml.gz',
 'https://apkpure.com/sitemaps/group-3.xml.gz',
 'https://apkpure.com/sitemaps/group-4.xml.gz',
 'https://apkpure.com/sitemaps/group-5.xml.gz',
 'https://apkpure.com/sitemaps/group-6.xml.gz',
 'https://apkpure.com/sitemaps/group-7.xml.gz',
 'https://apkpure.com/sitemaps/group-8.xml.gz',
 'https://apkpure.com/sitemaps/group-9.xml.gz',
 'https://apkpure.com/sitemaps/group-10.xml.gz',
 'https://apkpure.com/sitemaps/group-11.xml.gz',
 'https://apkpure.com/sitemaps/group-12.xml.gz',
 'https://apkpure.com/sitemaps/group-13.xml.gz',
 'https://apkpure.com/sitemaps/group-14.xml.gz',
 'https://apkpure.com/sitemaps/group-15.xml.gz

In [58]:
def gzLinkSort(gzlist):
    
    '''
    A method that returns a dictionary of gz urls based on there catagory.
    the key represents the catagory and the values are a list of urls that
    fall under the given catagory 
    

    gzlist - the list of gz files obtained from calling getSitemap
    '''
    
    
    firstPart = 'https://apkpure.com/sitemaps/'
    sep = '-'
    sep2 = '.'
    
    gzCat = defaultdict(list)
    links = []
    
    for url in gzlist:
        
        cat = url.split(firstPart, 1)[1]
        if '-' in cat:
            cat = cat.split(sep, 1)[0]     
        else:
            cat = cat.split(sep2, 1)[0]
        gzCat[cat].append(url)
        
    return gzCat
            
        
#x = gzLinkSort(gzlist)
#x

defaultdict(list,
            {'default': ['https://apkpure.com/sitemaps/default.xml.gz'],
             'topics': ['https://apkpure.com/sitemaps/topics.xml.gz'],
             'tag': ['https://apkpure.com/sitemaps/tag.xml.gz',
              'https://apkpure.com/sitemaps/tag-1.xml.gz',
              'https://apkpure.com/sitemaps/tag-2.xml.gz'],
             'group': ['https://apkpure.com/sitemaps/group.xml.gz',
              'https://apkpure.com/sitemaps/group-1.xml.gz',
              'https://apkpure.com/sitemaps/group-2.xml.gz',
              'https://apkpure.com/sitemaps/group-3.xml.gz',
              'https://apkpure.com/sitemaps/group-4.xml.gz',
              'https://apkpure.com/sitemaps/group-5.xml.gz',
              'https://apkpure.com/sitemaps/group-6.xml.gz',
              'https://apkpure.com/sitemaps/group-7.xml.gz',
              'https://apkpure.com/sitemaps/group-8.xml.gz',
              'https://apkpure.com/sitemaps/group-9.xml.gz',
              'https://apkpure.com/sit

In [109]:
def findLocs(outFile, path):
    
    '''
    A method that goes through the xml files of the catagories found in the getGzCats
    method and stores all the links in a text file. These links are links that contain
    the page where we can download the apk files. This method is intended to be a 
    helper method for getGzCats. 
    
    outFile - the xml file of the catagory to be taken in. This will then be searched through
    to find the links coresponding to the download pages of the apk
    
    path - the location you want your xml files to be downloaded to 
    
    '''
    
    file = open(outFile, 'r')
    line = file.readlines()

    locs = []
    for url in line:
        soup = BeautifulSoup(url, 'lxml')
        data = soup.find_all('loc')

        for i in data:
            locs.append(i.text)
    return locs

def getGzCats(gzDict, catagories, path):
    
    '''
    A method that takes in a dictionary (catagory (key), list of gz url (values))
    and the given catagories you wish to obtain the xml files of. The path specifies where
    the xml files will be downloaded. Do note that the xml files will be inserted into 
    a directory with the corresponding name of the catagory
    
    gzDict - a dictionary containing the catagories of gz files and their assosiated links
    
    catagories - the catagories of gz files you want to obtain the xml files for
    
    path - the location you want your xml files to be downloaded to 
    
    '''
    
    
    firstPart = 'https://apkpure.com/sitemaps/'
    os.chdir(path)
    
    for cat in catagories:
        
        os.chdir(path) 
        
        if not os.path.exists(cat):
            os.makedirs(cat)
            
        newPath = path + str(cat)
        os.chdir(newPath)
        
        for file in gzDict[cat]:
            
            print(cat)
            print(file)
            
            response = requests.get(file)
            compressed_file = io.BytesIO(response.content)
            decompressed_file = gzip.GzipFile(fileobj = compressed_file)
            out = str(cat) + '.xml'

            with open(out, 'wb') as outfile:
                outfile.write(decompressed_file.read())

            
            out2 = str(cat) + '_Links_APK' + '.txt'
            
            apklinks = findLocs(out, path)
            
            with open(out2, 'w') as outfile:
                outfile.write("\n".join(apklinks))
    

            
# getGzCats(x, ['comics'], '/Users/kelkin/Documents/DSC_180A/APK_Downloads/')

comics
https://apkpure.com/sitemaps/comics.xml.gz
comics
https://apkpure.com/sitemaps/comics-1.xml.gz
comics
https://apkpure.com/sitemaps/comics-2.xml.gz
comics
https://apkpure.com/sitemaps/comics-3.xml.gz
comics
https://apkpure.com/sitemaps/comics-4.xml.gz
comics
https://apkpure.com/sitemaps/comics-5.xml.gz
comics
https://apkpure.com/sitemaps/comics-6.xml.gz
comics
https://apkpure.com/sitemaps/comics-7.xml.gz
comics
https://apkpure.com/sitemaps/comics-8.xml.gz
comics
https://apkpure.com/sitemaps/comics-9.xml.gz
comics
https://apkpure.com/sitemaps/comics-10.xml.gz
comics
https://apkpure.com/sitemaps/comics-11.xml.gz
comics
https://apkpure.com/sitemaps/comics-12.xml.gz
comics
https://apkpure.com/sitemaps/comics-13.xml.gz
comics
https://apkpure.com/sitemaps/comics-14.xml.gz
comics
https://apkpure.com/sitemaps/comics-15.xml.gz
comics
https://apkpure.com/sitemaps/comics-16.xml.gz
comics
https://apkpure.com/sitemaps/comics-17.xml.gz
comics
https://apkpure.com/sitemaps/comics-18.xml.gz


In [110]:
def random_sampler(filename, k): 
    
    '''
    A method that randomly samples k number of apk urls from 
    a specified filename (ex. comics_Links_APK.txt)
    
    filename - the specified txt file to sample from
    
    k - the numebr of samples you wish to obtain for a given 
    catagory
    
    '''
    sample = []
    with open(filename, 'rb') as f:
        linecount = sum(1 for line in f)
        f.seek(0)

        random_linenos = sorted(random.sample(range(linecount), k), reverse = True)
        lineno = random_linenos.pop()
        for n, line in enumerate(f):
            if n == lineno:
                sample.append(line.rstrip())
                if len(random_linenos) > 0:
                    lineno = random_linenos.pop()
                else:
                    break
    return sample

# print(random_sampler('/Users/kelkin/Documents/DSC_180A/APK_Downloads/comics/comics_Links_APK.txt', 15))


[b'https://apkpure.com/wanted-prison-escape/com.enigmagamestudio.wanted.prison.escape', b'https://apkpure.com/free-jiocinema-advice-free-movies/com.musiczone.jiotv2', b'https://apkpure.com/tips-for-airtel-tv-free-guide-2019/airteltvtips.livetv.airteltv.tips.guide', b'https://apkpure.com/grand-train-robot-transformation/com.gp.grand.train.robot.transformation', b'https://apkpure.com/counter-terrorist-fps-army-shooting-2/com.survival.games.production.counter.comando.terrorist.fps.shooting.army.game', b'https://apkpure.com/funny-audios-jair-bolsonaro/bnctechnology.jairbolsonaro_audiosengracados', b'https://apkpure.com/%D8%A7%D9%84%D9%85%D8%AC%D9%8A%D8%A1-%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%89/com.andromo.dev651460.app1014406', b'https://apkpure.com/joker-prison-escape-2019/com.clown.prison.escape2019', b'https://apkpure.com/nilima/com.romel.nilima', b'https://apkpure.com/kkn-di-desa-penari-lengkap/com.andromo.dev786963.app1017326', b'https://apkpure.com/brawl-stars-quiz-2019-game/com.cartoo

In [116]:
# for url in strLocs:
#     site = requests.get(url).text
#     soup = BeautifulSoup(site, 'lxml')
    


def getDownloadLinks(catagories, num, path):
    
    '''
    Get the link to the downloads page for apk on the main page. 
    These links will direct us to the download page.
    
    num - specifies the number of links you wish to obtain 
    
    catagories - A list of the catagories of apps you wish to call from
    
    path - the location you want your xml files to be downloaded to 

    '''
    
    for cat in catagories:
        
        filename = str(cat) + '_Links_APK.txt'
        newPath = path + str(cat) + '/' + filename
        
        randomFiles = random_sampler(newPath, num)
        
        dLinks = []
        dic = {}

        for site in randomFiles:

            print(site)
            url = requests.get(site).text
            soup = BeautifulSoup(url, 'html.parser')
            try:
                apk = soup.find('div', attrs = {'class':'main page-q'}).find('div', attrs = {'class':'left'}).find('div', attrs = {'class':'box'}).find('dl', attrs = {'class':'ny-dl ny-dl-n'}).find('dd')
                apkWebsite = apk.find('div', attrs = {'class':'ny-down'}).find('a', attrs = {'class':'da'})
                name = apk.find('div', attrs = {'class':'title-like'}).find('h1')
                nameStr = str(name)[4:-5]
                downloadLink = apkWebsite.attrs['href']
                downloadLink = 'https://apkpure.com' + downloadLink
                dic[nameStr] = downloadLink
                dLinks.append(downloadLink)
                
                # Write sampled links to a text file 
                
                out = path + str(cat) + '/' + str(cat) + '_Download_Page_Links_APK' + '.txt'

                with open(out, 'w') as outfile:
                    outfile.write("\n".join(dLinks))
                    
                    
            except AttributeError as err:
                print(err)
                pass
    
    
    return dLinks
    
#getDownloadLinks(['beauty', 'comics'], 15, '/Users/kelkin/Documents/DSC_180A/APK_Downloads/')    

# # getDownloadLinks(10)

def downloadAPK(catagories, num, path):
    
    '''
    A method designed to get the links to download the APK of a given 
    android app. These links will download the APK file when entered into
    the browser 

    num - specifies the number of links you wish to obtain 
    
    catagories - A list of the catagories of apps you wish to call from
    
    path - the location you want your xml files to be downloaded to 
    
    **Current Limitations** - Some download links obtained from the getDownloadLinks
    method require the user to click on the download button and input a series of 
    characters to continue with their download. This prevent some websites from being 
    scrapped and the APK to be downloaded.  
    '''
    
    
    listAPK = []
    
    getDownloadLinks(catagories, num, path)
    
    for cat in catagories:
        
        filename = str(cat) + '_Download_Page_Links_APK.txt'
        newPath = path + str(cat) + '/' + filename
        
#         downloadPages = str(cat) + '_Links_APK.txt'
#         downloadPagesPath = path + str(cat) + '/' + downloadPages
        
        with open(newPath) as f:
            content = f.readlines()
        
        for site in content:

            names = []

            url = requests.get(site).text
            soup = BeautifulSoup(url, 'html.parser')

            try:
                apkDownload = soup.find('div', attrs = {'class':'main page-q'}).find('div', attrs = {'class':'left'}).find('div', attrs = {'class':'box'}).find('div', attrs = {'class':'fast-download-box fast-bottom'}).find('p', attrs = {'class':'down-click'}).find('a', attrs = {'id':'download_link'})
                listAPK.append(apkDownload.attrs['href'])
                names = apkDownload.attrs['title'][9:]
                
                # Write sampled links to a text file 
                
                out = path + str(cat) + '/' + str(cat) + '_Download_Links_APK' + '.txt'

                with open(out, 'w') as outfile:
                    outfile.write("\n".join(listAPK))
                    
            except AttributeError as err:
                print(err)
                pass
                


# def getNames(strLocs, num):
    
#     '''
#     A method designed to get the names to name the APK of a given 
#     android app.

#     num - specifies the number of links you wish to obtain 
#     starting at index 0
    
#     '''
    
#     listAPK = []
#     names = []
    
#     for site in getDownloadLinks(locs, num):
        
#         url = requests.get(site).text
#         soup = BeautifulSoup(url, 'html.parser')
        
#         try:
#             apkDownload = soup.find('div', attrs = {'class':'main page-q'}).find('div', attrs = {'class':'left'}).find('div', attrs = {'class':'box'}).find('div', attrs = {'class':'fast-download-box fast-bottom'}).find('p', attrs = {'class':'down-click'}).find('a', attrs = {'id':'download_link'})
#         except AttributeError as err:
#             print(err)
#             pass
#         else:
#             listAPK.append(apkDownload.attrs['href'])
#             names.append(apkDownload.attrs['title'][9:])
            
#     return names
        

# #downloadAPK(strLocs, 10)   

# # li = []

# # for i in soup.find_all('div', attrs = {'class':'main page-q'}):
# #     li.append(i)
# # li


# downloadAPK(['beauty', 'comics'], 15, '/Users/kelkin/Documents/DSC_180A/APK_Downloads/')

b'https://apkpure.com/pin-up/com.fexladusapps.trimmer'
b'https://apkpure.com/%EB%8C%80%EC%9E%90%EC%9D%B8-h-b/com.designhnb'
b'https://apkpure.com/karlav%C3%A4gen-21/se.bloop.karlavage'
b'https://apkpure.com/trainime-personal-fitness-trainer/com.gmail.nurazlina144.trainime'
b'https://apkpure.com/sabaya/com.emcan.sabaya'
b'https://apkpure.com/face-care/com.wazedsp.facecare'
b'https://apkpure.com/%D8%AA%D8%B1%D9%83%D9%8A%D8%A8-%D9%86%D8%B8%D8%A7%D8%B1%D8%A7%D8%AA-%D8%B4%D9%85%D8%B3%D9%8A%D8%A9-%D8%B1%D8%AC%D8%A7%D9%84%D9%8A%D8%A9/com.roujminax.mkp.xlabz_glassify'
b'https://apkpure.com/broken-heart-photo-frames/com.brokenheart.dualphotoframes'
b'https://apkpure.com/jv-barber-shop/br.com.jvbarbershop'
b'https://apkpure.com/barbearia-monteiro/br.com.barbeariamonteiro'
b'https://apkpure.com/maestria-barber-shop/br.com.maestriabarbershop'
b'https://apkpure.com/%D8%B5%D9%88%D8%B1-%D8%A8%D9%86%D8%A7%D8%AA-%D9%83%D9%8A%D9%88%D8%AA-%D8%AC%D8%AF%D9%8A%D8%AF%D8%A9-2019/com.banatcute.banat'
b'https:/

['https://download.apkpure.com/b/apk/Y29tLmVtY2FuLnNhYmF5YV84XzhjY2JkOTZl?_fn=U2FiYXlhINi12KjYp9mK2KdfdjEuMS44X2Fwa3B1cmUuY29tLmFwaw&k=f5e2bea4bfd247b270122e49125b60195e30ba01&as=65fb1e3baaaa7c03994717f506ed1b9c5e2e1779&_p=Y29tLmVtY2FuLnNhYmF5YQ&c=1%7CBEAUTY%7CZGV2PUVtY2FuJnQ9YXBrJnM9MTA3NTg5NDgmdm49MS4xLjgmdmM9OA',
 'https://download.apkpure.com/b/apk/Y29tLmJyb2tlbmhlYXJ0LmR1YWxwaG90b2ZyYW1lc18yX2I2OTQ2ZDQw?_fn=QnJva2VuIEhlYXJ0IFBob3RvIEZyYW1lc192MS4wMV9hcGtwdXJlLmNvbS5hcGs&k=a309afefc167d42935513571a144078c5e30ba04&as=d5f33c44d1c0aa8b1cd3b33845bb095c5e2e177c&_p=Y29tLmJyb2tlbmhlYXJ0LmR1YWxwaG90b2ZyYW1lcw&c=1%7CBEAUTY%7CZGV2PUJlYXV0eSUyMEFwcHMlMjBab25lJnQ9YXBrJnM9MTA5NTcxOTImdm49MS4wMSZ2Yz0y',
 'https://download.apkpure.com/b/xapk/Y29tLnBhbGxhZGl1bS5iZWF1dHkuZ2lybC5tYWNrdXBfMV8xNmI3NDEyZA?_fn=R2lybCBCZWF1dHkgSGFpcnN0eWxlIExpcHN0aWNrIEV5ZWJyb3dfdjEuMF9hcGtwdXJlLmNvbS54YXBr&k=be4efd052477c48351b995ec8c857bf35e30ba0a&as=38f4700312c3d2a88c4704d35b4e97ff5e2e1782&_p=Y29tLnBhbGxhZGl1bS5iZWF1d

In [129]:
def getAPK(catagories, path):
    
    '''
    A method that gets the downloaded apk files from apkpure and 
    stores them in their correct catagory directory in APK_Downloads. 
    This method then converts the downloaded apk files into smali code
    and stores it in the current catagory directory
    
    catagories - A list of the catagories of apps you wish to call from
    
    path - the location you want your xml files to be downloaded to 
    '''
    
    for cat in catagories:
        
        filename = str(cat) + '_Download_Links_APK.txt'
        newPath = path + str(cat) + '/' + filename
        catDir = path + str(cat)
        
        with open(newPath) as f:
            content = f.readlines()
            
        count = 0
        #nameList = getNames(strLocs, 10)
        
        for link in content:
            
            resp = requests.get(link, stream = True)
            contents = resp.content

            name = count
            
            out = os.path.join(catDir, str(cat) + '_' + str(name) +'.apk')
            #data = urllib2.urlopen(link).read()
            
            with open( out, "wb" ) as code:
                code.write(contents)
                os.chdir(catDir)
                os.system('apktool d ' + str(cat) + '_' + str(name) +'.apk') 

            count += 1
            
            
            
            
# getAPK(['beauty', 'comics'], '/Users/kelkin/Documents/DSC_180A/APK_Downloads/')




In [136]:
def get_data(**kwargs):

    path = kwargs['path']
    catagories = kwargs['catagories']
    sampleSize = kwargs['sampleSize']

    gzlist = getSitemap('https://apkpure.com/sitemap.xml')
    catDicGz = gzLinkSort(gzlist)
    getGzCats(catDicGz, catagories, path)
    downloadAPK(catagories, sampleSize, path)
    getAPK(catagories, path)


if __name__== "__main__":
    # NOTE: the following line of code is specific to where data-params-apk.json has been downloaded to 
    # os.chdir('/Users/kelkin/Documents/DSC_180A/APK_Downloads/')
    cfg = json.load(open('data-params-apk.json'))
    get_data(**cfg)

beauty
https://apkpure.com/sitemaps/beauty.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-1.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-2.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-3.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-4.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-5.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-6.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-7.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-8.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-9.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-10.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-11.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-12.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-13.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-14.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-15.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-16.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-17.xml.gz
beauty
https://apkpure.com/sitemaps/beauty-18.xml.gz
beaut

KeyboardInterrupt: 

In [ ]:
          
                
                
                

sample = random.sample(gzlist, 50)

for i in sample:

    response = requests.get(i)
    compressed_file = io.BytesIO(response.content)
    decompressed_file = gzip.GzipFile(fileobj = compressed_file)
    out = 'my_new_app.xml'

    with open(out, 'wb') as outfile:
        outfile.write(decompressed_file.read())
            

getXml(getSitemap('https://apkpure.com/sitemap.xml'))

In [68]:
firstPart = 'https://apkpure.com/sitemaps/'
file = 'https://apkpure.com/sitemaps/beauty-2.xml.gz'
z = file.split(firstPart, 1)[1][:-3]
z

    #cats = dic.value()
    cats = list(dic.values())
    for d in cats:
        if not os.path.exists(d):
            os.makedirs(d)
        getDownloadLinks(locs, 10)

'beauty-2.xml'

In [5]:
# def get_val(val): 
#     for key, value in getCatagory(sample).items():
# #         if val in value[i]: 
# #             return key 
  
#     return "key doesn't exist"

# get_key(['https://apkpure.com/sitemaps/music_and_audio-402.xml.gz'])

#getCatagory(sample).get('https://apkpure.com/sitemaps/game_role_playing-17.xml.gz')

In [6]:
sample[:10]

NameError: name 'sample' is not defined

In [7]:
def getCatagory(gzFiles):
    
    '''
    A method that obtains all the possible catagories that 
    have been collected from the sample of gz files and urls. 
    
    This method will return a dictionary of catagory and gz file key 
    value pairs. 
    
    gzFiles - a list of the gz file urls on the sitemap
    '''
    
    catFileDic = dict()
    catagories = []
    for gz in gzFiles:
        key = gz
        catFileDic[key] = gz[29:-7].split('-')[0]
        catagories.append(key)
    
    return catFileDic


def catagoryDirectories(dic, path):
    
    '''
    A method that creates directories coresponding to the 
    catagory name obtained in the getCatagory method. 
    
    dic - the dictionary of key value pairs of catagory and gz urls
    
    path - the absolute path of the directory where we want to create
    the catagory directories 
    '''
    
    os.chdir(path)
    #cats = dic.value()
    cats = list(dic.values())
    for d in cats:
        if not os.path.exists(d):
            os.makedirs(d)
        getDownloadLinks(locs, 10)
    
    
#catagoryDirectories(getCatagory(sample), '/Users/kelkin/Documents/DSC_180A/APK_Downloads')

In [8]:
file = open(out, 'r')
line = file.readlines()

locs = []
for url in line:
    soup = BeautifulSoup(url, 'lxml')
    data = soup.find_all('loc')

    for i in data:
        locs.append(i.text)
# i.text 

NameError: name 'out' is not defined

In [9]:
type(locs[0])
locs

NameError: name 'locs' is not defined

In [10]:
# Converted list of bs4.element.Tag to list of string URL without loc tags

# strLocs = []

# for i in locs:
#     strLocs.append(str(i))
# strLocs = [s[5:-6] for s in strLocs]
# strLocs

In [90]:
#getCatagory(sample)

In [11]:
# for url in strLocs:
#     site = requests.get(url).text
#     soup = BeautifulSoup(site, 'lxml')
    


def getDownloadLinks(locs, num):
    
    '''
    Get the link to the downloads page for apk on the main page. 
    These links will direct us to the download page.

    num - specifies the number of links you wish to obtain 
    starting at index 0
    '''
    dLinks = []
    dic = {}

    for site in locs[:num]:
        
        url = requests.get(site).text
        soup = BeautifulSoup(url, 'html.parser')
        
        apk = soup.find('div', attrs = {'class':'main page-q'}).find('div', attrs = {'class':'left'}).find('div', attrs = {'class':'box'}).find('dl', attrs = {'class':'ny-dl ny-dl-n'}).find('dd')
        apkWebsite = apk.find('div', attrs = {'class':'ny-down'}).find('a', attrs = {'class':'da'})
        name = apk.find('div', attrs = {'class':'title-like'}).find('h1')
        nameStr = str(name)[4:-5]
        
        downloadLink = apkWebsite.attrs['href']
        downloadLink = 'https://apkpure.com' + downloadLink
        dic[nameStr] = downloadLink
        dLinks.append(downloadLink)
        
    return dLinks
    
# getDownloadLinks(10)

def downloadAPK(strLocs, num):
    
    '''
    A method designed to get the links to download the APK of a given 
    android app.

    num - specifies the number of links you wish to obtain 
    starting at index 0
    
    **Current Limitations** - Some download links obtained from the getDownloadLinks
    method require the user to click on the download button and input a series of 
    characters to continue with their download. This prevent some websites from being 
    scrapped and the APK to be downloaded 
    '''
    listAPK = []
    
    for site in getDownloadLinks(locs, num):
        
        names = []
        
        url = requests.get(site).text
        soup = BeautifulSoup(url, 'html.parser')
        
        try:
            apkDownload = soup.find('div', attrs = {'class':'main page-q'}).find('div', attrs = {'class':'left'}).find('div', attrs = {'class':'box'}).find('div', attrs = {'class':'fast-download-box fast-bottom'}).find('p', attrs = {'class':'down-click'}).find('a', attrs = {'id':'download_link'})
        except AttributeError as err:
            print(err)
            pass
        else:
            listAPK.append(apkDownload.attrs['href'])
            print(apkDownload.attrs['href'])
            print(apkDownload.attrs['title'][9:])
            names = apkDownload.attrs['title'][9:]
            
    return listAPK


def getNames(strLocs, num):
    
    '''
    A method designed to get the names to name the APK of a given 
    android app.

    num - specifies the number of links you wish to obtain 
    starting at index 0
    
    '''
    
    listAPK = []
    names = []
    
    for site in getDownloadLinks(locs, num):
        
        url = requests.get(site).text
        soup = BeautifulSoup(url, 'html.parser')
        
        try:
            apkDownload = soup.find('div', attrs = {'class':'main page-q'}).find('div', attrs = {'class':'left'}).find('div', attrs = {'class':'box'}).find('div', attrs = {'class':'fast-download-box fast-bottom'}).find('p', attrs = {'class':'down-click'}).find('a', attrs = {'id':'download_link'})
        except AttributeError as err:
            print(err)
            pass
        else:
            listAPK.append(apkDownload.attrs['href'])
            names.append(apkDownload.attrs['title'][9:])
            
    return names
        

#downloadAPK(strLocs, 10)   

# li = []

# for i in soup.find_all('div', attrs = {'class':'main page-q'}):
#     li.append(i)
# li

In [12]:
getNames(strLocs, 10)

NameError: name 'strLocs' is not defined

In [13]:
# # for url in strLocs:
# #     site = requests.get(url).text
# #     soup = BeautifulSoup(site, 'lxml')
    
# dLinks = []

# #Get the link to the downloads page for apk on the main page. These links will direct us to the download page 

# for site in strLocs[:10]:
#     url = requests.get(site).text
#     soup = BeautifulSoup(url, 'html.parser')
#     apkWebsite = soup.find('div', attrs = {'class':'main page-q'}).find('div', attrs = {'class':'left'}).find('div', attrs = {'class':'box'}).find('dl', attrs = {'class':'ny-dl ny-dl-n'}).find('dd').find('div', attrs = {'class':'ny-down'}).find('a', attrs = {'class':'da'})
#     downloadLink = apkWebsite.attrs['href']
#     downloadLink = 'https://apkpure.com' + downloadLink
#     dLinks.append(downloadLink)
# dLinks



def getAPK(links):
    
    '''
    A method that gets the downloaded apk files from apkpure and 
    stores them in a directory called APK_Downloads
    
    links - the URL's where the apk files will begin their download
    '''
    
    count = 0
    nameList = getNames(strLocs, 10)
    for link in links:
        resp = requests.get(link, stream = True)
        contents = resp.content
        
        name = nameList[count]
        
        directory = os.path.abspath('/Users/kelkin/Documents/DSC_180A/APK_Downloads')
        out = os.path.join(directory, str(name)+'.apk')
        #data = urllib2.urlopen(link).read()
        
        with open( out, "wb" ) as code:
            code.write(contents)
            
        count += 1
        
        
getAPK(downloadAPK(strLocs, 10))




# def createDirectories():
    
#     '''
#     A method designed to create directories inside the APK_Downloads Directory
#     to store the downloads obtained from apkpure. 
    
#     *Note: this method will use the dictionary returned from getDownloadLinks to 
#     specify the name of the directory and download link
#     '''
    
#     path = "/Users/kelkin/Documents/DSC_180A/APK_Downloads"

#     try:
#         os.mkdir(path)
#     except OSError:
#         print ("Creation of the directory %s failed" % path)
#     else:
#         print ("Successfully created the directory %s " % path)
    
    






NameError: name 'strLocs' is not defined

In [34]:
def getSmaliCode(APKs):
    
    '''
    A method that takes in a directory where the apk files
    are contained. This method will then call apktool to 
    obtain the smali code for each apk file
    
    APKs - the directory where the apk files can be found 
    '''
    
    os.chdir(APKs)
    for filename in os.listdir(APKs):
        os.system('apktool d ' + filename)
        
getSmaliCode('/Users/kelkin/Documents/DSC_180A/APK_Downloads')

In [31]:
# os.chdir('/Users/kelkin/Documents/DSC_180A')
# cwd = os.getcwd()

# cwd


'/Users/kelkin/Documents/DSC_180A'

In [ ]:
PATH : where to output files
Catagories: catagory of gz files to get 
Number of links from each GZ :

In [ ]:
def config(**kwargs):
    jsonDic = {}
    kwargs['cat'] 
    

In [122]:
if __name__ == 'main':
    conf = json.loads(open('config.json'))
    config(**conf)
    
    
    getXml(getSitemap('https://apkpure.com/sitemap.xml'))

SyntaxError: unexpected EOF while parsing (<ipython-input-122-31ae4e27ff6b>, line 1)

In [ ]:
{
    'cat':'books'
}